# A09 - [Project: Street or Road?](https://dtkaplan.github.io/DataComputingEbook/project-street-or-road.html#project-street-or-road)
Kaplan, Daniel & Matthew Beckman. (2021). _Data Computing_. 2nd Ed. [Home](https://dtkaplan.github.io/DataComputingEbook/).

https://davefriedman01.github.io/Mathematics/computer/program/rlang/STAT184/intro.html

---

```{admonition} Revised
20 Jun 2023
```
```{contents}
```

---

## Programming Environment

In [2]:
library(lubridate)
library(rvest)
library(tidyverse)

str_c('EXECUTED : ', now())
sessionInfo()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.2     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.2     ✔ tibble  3.2.1
✔ purrr   1.0.1     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "EXECUTED : 2023-06-20 01:39:27.302263"

R version 4.3.0 (2023-04-21)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS Ventura 13.3.1

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] forcats_1.0.0   stringr_1.5.0   dplyr_1.1.2     purrr_1.0.1    
 [5] readr_2.1.4     tidyr_1.3.0     tibble_3.2.1    ggplot2_3.4.2  
 [9] tidyverse_2.0.0 rvest_1.0.3     lubridate_1.9.2

loaded via a namespace (and not attached):
 [1] gtable_0.3.3     jsonlite_1.8.5   compiler_4.3.0   crayon_1.5.2    
 [5] tidyselect_1.2.0 IRdisplay_1.1    xml2_1.3.4       scales_1.2.1    
 [9] uuid_1.1-0       fastma

---

People's addresses involve streets, lanes, courts, avenues, and so on. How many such road-related words are in common use? In answering this question, you would presumably want to look at lots of addresses and extract the road-related term. You could do this by eye, reading down a list of a few hundred or thousand addresses. But if you want to do it on a really large scale, a city or state or country, you would want some automated help, for instance, a computer program that discards the sorts of entries you have already identified to give a greater concentration of unidentified terms. In this activity, you're going to build such a program.

In [4]:
# about 15,000 street addresses of registered voters in Wake County, NC
Addresses <- read_csv('https://mdbeckman.github.io/dcSupplement/data/street-addresses.csv')
head(Addresses)

Rows: 15483 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): address

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


address
<chr>
2117 MARINER CIRCLE
101 EPPING WAY
PO BOX 58592
5102 ECHO RIDGE RD
PO BOX 37218
PO BOX 37218


In [6]:
# about 900,000 medicare service provider street addresses
download.file(url      = 'https://mdbeckman.github.io/dcSupplement/data/CMS_ProvidersSimple.rds',
              destfile = 'CMS_ProvidersSimple.rds')
DataTable <- readRDS('CMS_ProvidersSimple.rds')
head(DataTable)

,address,first_name,sex
,<chr>,<chr>,<chr>
1,900 SETON DR,ARDALAN,M
2,2650 RIDGE AVE,THOMAS,M
3,4126 N HOLLAND SYLVANIA RD,RASHID,M
4,456 MAGEE AVE,DAVID,M
5,11100 EUCLID AVE,JENNIFER,F
6,12605 E 16TH AVE,KEVIN,M


To solve such problems, start by looking at a few dozen of the addresses to familiarize yourself with common patterns. Suppose you wanted to extract the PO Box number from an address. Read the street address data and pull out a sample of a few dozen cases.
1. In everyday langauge, describe a pattern that you think will identify the information you are looking for.
    * The PO Box cases tend to have a substring 'PO'.
2. Translate (1) into the form of a regular expression.
    * The regular expression for 'PO' is simply 'PO'.
3. Filter to retain the cases that match the expression. Hint: `filter()` and `grepl()` are useful for this.
4. Filter to retain the cases that do not match the expression.
5. Examine the results of (3) and (4) to identify shortcomings in your patterns.
6. Improve or extend the pattern to deal with the mistaken cases.
7. Repeat until satisfied.
8. Put extraction parentheses around the parts of the regular expression that contain the info you want.

In [9]:
Sample <-
  Addresses %>%
    sample_n(size = 50)
Sample

address
<chr>
PO BOX 581
P O BOX 259
NCSU BOX 21687
6216 LULA RIDGE ST
NCSU BOX 04597
5008 WHISPER CT
9660 FALLS OF NEUSE STE138-PMB445
1000 SHEPHERDS SCHOOL RD
NCSU BOX 03375


In [11]:
Matches <-
  Sample %>%
    filter(grepl(pattern = 'PO', address))
Matches

address
<chr>
PO BOX 581
PO BOX 330
PO BOX 99567
PO BOX 575
PO BOX 1295
PO BOX 33201
PO BOX 334
PO BOX 692
PO BOX 1031


In [12]:
Dont <-
  Sample %>%
    filter(!grepl(pattern = 'PO', address))
Dont

address
<chr>
P O BOX 259
NCSU BOX 21687
6216 LULA RIDGE ST
NCSU BOX 04597
5008 WHISPER CT
9660 FALLS OF NEUSE STE138-PMB445
1000 SHEPHERDS SCHOOL RD
NCSU BOX 03375
5509 LAKE EDGE DR


In [14]:
pattern <- 'BOX\\s+(\\d+)'
Matches <-
  Sample %>%
    filter( grepl(pattern, address))
Dont    <-
  Sample %>%
    filter(!grepl(pattern, address))

In [15]:
BoxNumbers <-
  Sample %>%
    filter(grepl(pattern, address)) %>%
    tidyr::extract(address, into = 'boxnum', regex = pattern)
BoxNumbers

boxnum
<chr>
581
259
21687
04597
03375
330
99567
575
1295


---